### This notebook outputs beer_reviews_clean into a clean_data folder in the machine learning branch
### The outputted .csv file provides average metrics of all the reviews for each beer as well as a 'review_count' column that lists how many reviews of each beer were included in the original dataset.



In [1]:

# Import dependencies
import pandas as pd

In [37]:
# read in the dataset
beer_df = pd.read_csv("../../Final_Project/beer_reviews.csv")
# beer_df.head()
beer_df.dtypes


brewery_id              int64
brewery_name           object
review_time             int64
review_overall        float64
review_aroma          float64
review_appearance     float64
review_profilename     object
beer_style             object
review_palate         float64
review_taste          float64
beer_name              object
beer_abv              float64
beer_beerid             int64
dtype: object

In [38]:
# Count the rows
len(beer_df)


1586614

In [39]:
# Change the review_time column to datetime
beer_df["review_time"] = pd.to_datetime(beer_df["review_time"], unit = 's' )
beer_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   brewery_id          1586614 non-null  int64         
 1   brewery_name        1586599 non-null  object        
 2   review_time         1586614 non-null  datetime64[ns]
 3   review_overall      1586614 non-null  float64       
 4   review_aroma        1586614 non-null  float64       
 5   review_appearance   1586614 non-null  float64       
 6   review_profilename  1586266 non-null  object        
 7   beer_style          1586614 non-null  object        
 8   review_palate       1586614 non-null  float64       
 9   review_taste        1586614 non-null  float64       
 10  beer_name           1586614 non-null  object        
 11  beer_abv            1518829 non-null  float64       
 12  beer_beerid         1586614 non-null  int64         
dtypes: datetime6

In [40]:
# Count null values
beer_df.isnull().sum()

brewery_id                0
brewery_name             15
review_time               0
review_overall            0
review_aroma              0
review_appearance         0
review_profilename      348
beer_style                0
review_palate             0
review_taste              0
beer_name                 0
beer_abv              67785
beer_beerid               0
dtype: int64

In [41]:
# How much of the dataset is null?
percent_null = beer_df.isnull().sum()/len(beer_df) *100
print(percent_null)


brewery_id            0.000000
brewery_name          0.000945
review_time           0.000000
review_overall        0.000000
review_aroma          0.000000
review_appearance     0.000000
review_profilename    0.021934
beer_style            0.000000
review_palate         0.000000
review_taste          0.000000
beer_name             0.000000
beer_abv              4.272306
beer_beerid           0.000000
dtype: float64


In [42]:
# At worst, we're only losing less than 4.5% of the dataset by dropping rows with null values
beer_df = beer_df.dropna()
beer_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1518478 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   brewery_id          1518478 non-null  int64         
 1   brewery_name        1518478 non-null  object        
 2   review_time         1518478 non-null  datetime64[ns]
 3   review_overall      1518478 non-null  float64       
 4   review_aroma        1518478 non-null  float64       
 5   review_appearance   1518478 non-null  float64       
 6   review_profilename  1518478 non-null  object        
 7   beer_style          1518478 non-null  object        
 8   review_palate       1518478 non-null  float64       
 9   review_taste        1518478 non-null  float64       
 10  beer_name           1518478 non-null  object        
 11  beer_abv            1518478 non-null  float64       
 12  beer_beerid         1518478 non-null  int64         
dtypes: datetime6

In [43]:
# A very small percentage (7 out of 1.5 million) of beers have a 0 rating, so these can be dropped.
beer_df = beer_df[beer_df.review_overall != 0]

In [44]:
# Check for and count duplicate rows
len(beer_df)-len(beer_df.drop_duplicates())

0

In [45]:
# Check to see if a beer has been reviewed more than once by the same reviewer
len(beer_df) - len(beer_df.drop_duplicates(subset = ["review_profilename", "beer_beerid"]))
# Drop duplicate rows where a reviewer reviewed a beer more than once
# We want to keep the most recent score of the beer
beer_df = beer_df.sort_values("review_time", ascending = False)
beer_df = beer_df.drop_duplicates(subset = ["review_profilename", "beer_beerid"])


In [46]:
# Count of different number of beers and beer styles reviewed
beer_df.nunique(axis =0)

brewery_id               5230
brewery_name             5155
review_time           1496139
review_overall              9
review_aroma                9
review_appearance           9
review_profilename      32908
beer_style                104
review_palate               9
review_taste                9
beer_name               44075
beer_abv                  530
beer_beerid             49000
dtype: int64

In [47]:
# group data by beer and brewery
beer = beer_df.groupby(["beer_name","brewery_name","beer_style"])

In [48]:
# Create a data frame for average rating for each beer
# Set index to brewery name and beer style
beer_rating_df = pd.DataFrame(beer.mean())
ind_beer_rating_df = beer_rating_df.reset_index(["brewery_name", "beer_style"])

# Change the id column back to an integer for merging later
ind_beer_rating_df.beer_beerid = ind_beer_rating_df.beer_beerid.astype(int)

ind_beer_rating_df.head()

,brewery_name,beer_style,brewery_id,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
beer_name,,,,,,,,,,
! (Old Ale),Närke Kulturbryggeri AB,Old Ale,10902.0,4.0,4.5,4.00,4.000000,4.5,8.2,57645
"""100""",The Covey Restaurant & Brewery,Belgian Dark Ale,13338.0,4.0,4.0,4.00,4.000000,4.4,10.0,49236
"""100"" Pale Ale",Aviator Brewing Company,American IPA,18635.0,4.0,4.0,3.50,4.000000,4.0,6.6,50696
"""12"" Belgian Golden Strong Ale",Flossmoor Station Restaurant & Brewery,Belgian Strong Pale Ale,612.0,4.0,3.5,4.25,4.750000,4.5,9.0,44353
"""33"" Export",Baltika-Baku,Dortmunder / Export Lager,13640.0,3.0,3.0,3.00,3.166667,3.0,4.8,30080


In [49]:
# Save the number of reviews of each beer
beer_review_count = beer_df.groupby(['beer_name', 'beer_beerid']).count()
beer_review_count = beer_review_count[['review_overall']]
beer_review_count = beer_review_count.reset_index()
beer_review_count.head()

,beer_name,beer_beerid,review_overall
0,! (Old Ale),57645,1
1,"""100""",49236,5
2,"""100"" Pale Ale",50696,1
3,"""12"" Belgian Golden Strong Ale",44353,2
4,"""33"" Export",30080,3


In [50]:
# Put together average rating and number of reviews
beer_reviews_clean = pd.merge(ind_beer_rating_df, beer_review_count, on='beer_beerid', how='outer')
beer_reviews_clean = beer_merge.rename(columns={'review_overall_x': 'review_overall', 'review_overall_y': 'review_count'})
beer_reviews_clean.head()

,brewery_name,beer_style,brewery_id,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid,beer_name,review_count
0,Närke Kulturbryggeri AB,Old Ale,10902.0,4.0,4.5,4.00,4.000000,4.5,8.2,57645,! (Old Ale),1.0
1,The Covey Restaurant & Brewery,Belgian Dark Ale,13338.0,4.0,4.0,4.00,4.000000,4.4,10.0,49236,"""100""",5.0
2,Aviator Brewing Company,American IPA,18635.0,4.0,4.0,3.50,4.000000,4.0,6.6,50696,"""100"" Pale Ale",1.0
3,Flossmoor Station Restaurant & Brewery,Belgian Strong Pale Ale,612.0,4.0,3.5,4.25,4.750000,4.5,9.0,44353,"""12"" Belgian Golden Strong Ale",2.0
4,Baltika-Baku,Dortmunder / Export Lager,13640.0,3.0,3.0,3.00,3.166667,3.0,4.8,30080,"""33"" Export",3.0


In [51]:
beer_reviews_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49141 entries, 0 to 49140
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   brewery_name       48832 non-null  object 
 1   beer_style         48832 non-null  object 
 2   brewery_id         48832 non-null  float64
 3   review_overall     48832 non-null  float64
 4   review_aroma       48832 non-null  float64
 5   review_appearance  48832 non-null  float64
 6   review_palate      48832 non-null  float64
 7   review_taste       48832 non-null  float64
 8   beer_abv           48832 non-null  float64
 9   beer_beerid        49141 non-null  int32  
 10  beer_name          49074 non-null  object 
 11  review_count       49074 non-null  float64
dtypes: float64(8), int32(1), object(3)
memory usage: 4.7+ MB


In [52]:
beer_reviews_clean = beer_reviews_clean.dropna()
beer_reviews_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48765 entries, 0 to 48831
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   brewery_name       48765 non-null  object 
 1   beer_style         48765 non-null  object 
 2   brewery_id         48765 non-null  float64
 3   review_overall     48765 non-null  float64
 4   review_aroma       48765 non-null  float64
 5   review_appearance  48765 non-null  float64
 6   review_palate      48765 non-null  float64
 7   review_taste       48765 non-null  float64
 8   beer_abv           48765 non-null  float64
 9   beer_beerid        48765 non-null  int32  
 10  beer_name          48765 non-null  object 
 11  review_count       48765 non-null  float64
dtypes: float64(8), int32(1), object(3)
memory usage: 4.7+ MB


In [53]:
beer_reviews_clean.sort_values("review_count", ascending = False)

,brewery_name,beer_style,brewery_id,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid,beer_name,review_count
731,Dogfish Head Brewery,American Double / Imperial IPA,10099.0,4.146912,4.213818,4.191672,4.179975,4.327667,9.0,2093,90 Minute IPA,3206.0
31856,North Coast Brewing Co.,Russian Imperial Stout,112.0,4.173963,4.199144,4.372120,4.229427,4.342495,9.0,412,Old Rasputin Russian Imperial Stout,3038.0
39547,Sierra Nevada Brewing Co.,American IPA,140.0,4.167805,4.081427,4.224821,4.074599,4.187948,6.8,1904,Sierra Nevada Celebration Ale,2929.0
41818,Stone Brewing Co.,American Double / Imperial IPA,147.0,4.163389,4.342852,4.180598,4.183245,4.350983,7.7,4083,Stone Ruination IPA,2644.0
2791,Stone Brewing Co.,American Strong Ale,147.0,4.079947,4.119445,4.295480,4.144132,4.279719,7.2,92,Arrogant Bastard Ale,2633.0
...,...,...,...,...,...,...,...,...,...,...,...,...
24671,Kraftbräu Brewery,German Pilsener,2725.0,3.500000,3.500000,4.000000,3.000000,4.000000,5.0,25344,Kraftbräu Hoppity Boop Lager,1.0
24667,Kraftbräu Brewery,Vienna Lager,2725.0,4.000000,4.000000,3.500000,4.000000,4.000000,5.5,6449,Kraftbräu Dark Bohemian,1.0
24665,Kraftbräu Brewery,Dortmunder / Export Lager,2725.0,4.000000,4.000000,4.000000,4.000000,4.000000,6.0,19912,Kraftbräu Celery City Dortmunder Export,1.0
24661,Kraft Bräu - Blesius Garten,Hefeweizen,4494.0,3.500000,3.500000,3.500000,3.500000,4.000000,4.9,61888,Kraft Brau Weizen,1.0


In [54]:
print (len(beer_reviews_clean[beer_reviews_clean["review_count"]>5]))


17111


## Do we want to keep all beers?  Only those with more than a certain number of reviews?
### There are 17,111 beers with fewer than 5 reviews.  Maybe that doesn't matter?  Maybe we need to wait and see how we're able to use the data.

In [55]:
# Save the cleaned beer review dataset
beer_reviews_clean.to_csv("Clean_data/beer_reviews_clean.csv")